<a href="https://colab.research.google.com/github/reddevil-ux/AI/blob/main/HOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q python-telegram-bot transformers torch Pillow nest_asyncio > /dev/null 2>&1

import nest_asyncio
nest_asyncio.apply()

import logging
import torch
import random
import re
import asyncio
from PIL import Image
from io import BytesIO
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes, CallbackQueryHandler
from transformers import BlipProcessor, BlipForConditionalGeneration, GPT2LMHeadModel, GPT2Tokenizer

a = "8484962267:AAFfAoLvBWdzWd8UJtW8aW0BLNoH0LmDHf8"

logging.basicConfig(level=logging.WARNING)

b = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
c = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
d = GPT2Tokenizer.from_pretrained("distilgpt2")
e = GPT2LMHeadModel.from_pretrained("distilgpt2")

f = torch.device("cuda" if torch.cuda.is_available() else "cpu")
c = c.to(f)
e = e.to(f)

d.pad_token = d.eos_token

g = {}

h = [
    "A large group of people enjoying dinner together at a busy restaurant with red chairs.",
    "The sun setting behind tall mountains casting long shadows across the valley below.",
    "Someone sitting on a wooden bench reading a thick book on a sunny afternoon.",
    "A fluffy orange cat sitting quietly on a windowsill watching birds outside.",
    "A blue car driving down a wet road after a heavy rainstorm.",
    "Young children laughing and playing soccer in a park with green grass.",
    "Two people walking hand in hand along a sandy beach at sunset.",
    "A person wearing an apron while cooking pasta in a modern kitchen.",
    "A brown dog excitedly chasing a red ball in the backyard.",
    "Bright city lights reflecting off buildings during a rainy night.",
    "Colorful flowers blooming in a well-maintained garden near a fountain.",
    "A person sitting at a desk typing on a laptop with coffee nearby.",
    "A family of four sitting together watching television in their living room.",
    "Several small birds flying together against a clear blue sky.",
    "A white sailboat floating peacefully on calm blue water.",
    "Raindrops falling on gray pavement creating small puddles on the ground.",
    "Fresh snow covering the branches of tall pine trees in winter.",
    "Someone wearing a helmet riding a bicycle down a quiet street.",
    "A small store with many items displayed neatly on wooden shelves.",
    "An empty classroom with desks arranged in rows and a chalkboard.",
    "A tourist taking a picture of a famous landmark with a camera.",
    "A man dressed in a formal suit and tie at a business meeting.",
    "A woman holding a paper coffee cup while walking down the street.",
    "A young child playing with colorful plastic toys on the floor.",
    "A person lifting weights at a gym filled with exercise equipment.",
    "A teacher pointing at a whiteboard during a classroom lesson.",
    "A person waiting at a bus stop looking at their phone.",
    "A shopper carrying several bags while walking through a mall.",
    "A person talking on the phone while walking in the city.",
    "A man walking a small white dog on a leash in the park.",
    "Commuters hurrying through a busy train station during evening rush hour.",
    "A couple sitting together on a park bench feeding pigeons nearby.",
    "Children running and laughing while playing tag in a playground.",
    "Someone sleeping peacefully on a comfortable couch with a blanket.",
    "A person typing quickly on a laptop at a coffee shop table.",
    "Rain falling on a colorful umbrella held by a waiting pedestrian.",
    "A small bird perched on a tree branch singing in the morning.",
    "A large window offering a stunning view of the city skyline.",
    "People sitting on a checkered blanket having a picnic in the park.",
    "A person jogging along a path in the park on a sunny day.",
    "Someone sitting quietly drinking tea from a ceramic cup.",
    "A messy desk covered with papers, books, and office supplies.",
    "A clean and tidy room with everything organized in its place.",
    "A person gazing thoughtfully out a window at the rain.",
    "Two friends sitting at a cafe table talking and laughing together.",
    "A person working diligently in a modern office with computers.",
    "A person standing at a bus stop checking the time on their watch.",
    "A tired dog lying on the cool floor resting after a walk.",
    "A cat curled up sleeping in a cozy wicker basket.",
    "Shoppers browsing through a busy outdoor market on weekend morning.",
    "A person holding a steaming cup of coffee on a cold day.",
    "Someone completely absorbed in their smartphone while walking.",
    "A person waiting impatiently while checking their phone repeatedly.",
    "A quiet library with students reading books at wooden tables.",
    "A crowded cafeteria during lunch hour with people eating and talking.",
    "A person wearing a bright colorful shirt standing out in a crowd.",
    "Someone with a serious expression staring intently at something.",
    "A person smiling happily while receiving good news on the phone.",
    "Two people engaged in a heated argument with expressive gestures.",
    "A person looking confused while studying a complicated map.",
    "Someone checking their wristwatch anxiously as they wait.",
    "A person carrying multiple shopping bags struggling with the weight.",
    "Someone taking a selfie with friends in front of a landmark.",
    "A person sitting down to put on their shoes before going out.",
    "Someone bending over to tie their shoelaces on the sidewalk.",
    "A person reaching out to open a heavy wooden door.",
    "Someone carefully closing a window to keep out the cold.",
    "A person sitting alone on a bench watching people pass by.",
    "Someone standing under a large tree seeking shade from the sun.",
    "A person leaning casually against a brick wall waiting.",
    "Someone walking carefully down a steep flight of stairs.",
    "A person riding an escalator while looking at their phone.",
    "Someone pushing a baby stroller along a paved pathway.",
    "A person holding an umbrella against the wind and rain.",
    "Someone wearing a heavy backpack while hiking on a trail.",
    "A person wearing stylish sunglasses on a bright sunny day.",
    "Someone wearing a fashionable hat while walking in the city.",
    "A person dressed in a long coat on a cold winter day.",
    "Someone wearing casual jeans and a t-shirt on a weekend.",
    "A person dressed in elegant formal wear attending an event.",
    "Someone wearing athletic clothes and running shoes for exercise.",
    "A man with a neatly trimmed beard reading a newspaper.",
    "A woman with long flowing hair walking in the wind.",
    "A person wearing glasses pushing them up while reading.",
    "Someone with a visible tattoo on their arm showing it proudly.",
    "A person holding a book while waiting in line at a store.",
    "Someone reading a newspaper while sitting on a park bench.",
    "A tourist holding a camera ready to take the perfect shot.",
    "Someone studying a map carefully trying to find their way.",
    "A traveler carrying a large suitcase through the airport.",
    "A student with a backpack walking to class on campus.",
    "A person gently holding a flower and smelling its fragrance.",
    "A child holding a bright balloon at a birthday party.",
    "A person walking a small energetic dog pulling on the leash.",
    "Someone struggling to control a large powerful dog on a walk.",
    "A person sitting on a bench throwing breadcrumbs to birds.",
    "Someone gazing up at the sky watching clouds pass by.",
    "A person watching the sunset colors spread across the horizon.",
    "Someone staring at the ground deep in thought about something.",
    "A person looking at their own reflection in a store window.",
    "Someone checking their appearance in a small pocket mirror.",
    "A person adjusting their clothing before entering a building.",
    "Someone fixing their hair that got messed up by the wind.",
    "A person yawning widely after a long tiring day at work.",
    "Someone stretching their arms above their head to relieve tension.",
    "A person rubbing their tired eyes late at night.",
    "Someone looking exhausted while waiting for the bus.",
    "A person with a surprised expression witnessing something unexpected.",
    "Someone looking shocked at news on their phone screen.",
    "A person with a confused expression trying to understand.",
    "Someone appearing thoughtful while staring into the distance.",
    "A person with a determined look ready to face a challenge.",
    "Someone looking worried while checking the time repeatedly.",
    "A person with a sad expression after receiving bad news.",
    "Someone looking genuinely happy and content with life.",
    "A person with an excited look about to do something fun.",
    "Someone appearing bored while waiting in a long line.",
    "A person with an angry expression arguing with someone.",
    "Someone looking frustrated while dealing with a difficult problem.",
    "A person with a calm expression meditating in a peaceful place.",
    "Someone looking completely relaxed on a comfortable chair."
]

async def i(j: Update, k: ContextTypes.DEFAULT_TYPE):
    l = "Отправь фото. Я покажу 5 вариантов описания. Выбери правильный."
    await j.message.reply_text(l)

async def m(n: Update, o: ContextTypes.DEFAULT_TYPE):
    p = "Отправь фото → Получи 5 вариантов → Выбери правильный."
    await n.message.reply_text(p)

async def q(r: Update, s: ContextTypes.DEFAULT_TYPE):
    t = "Игра: 1 правда, 4 выдумки. Найди правду."
    await r.message.reply_text(t)

def u(v):
    w = Image.open(BytesIO(v))
    if w.mode != "RGB":
        w = w.convert("RGB")
    w.thumbnail((400, 400))
    x = b(images=w, return_tensors="pt").to(f)
    with torch.no_grad():
        y = c.generate(**x, max_length=30, num_beams=3)
    z = b.decode(y[0], skip_special_tokens=True)
    aa = z.split()
    if aa:
        aa[0] = aa[0].capitalize()
        z = ' '.join(aa)
        if not z.endswith('.'):
            z = z + '.'
    return z

def ac(ad, ae=4):
    af = []
    ag = len(ad.split())
    ah = []
    for ai in h:
        if ai.lower() != ad.lower():
            aj = len(ai.split())
            if abs(aj - ag) <= 7:
                ah.append(ai)

    random.shuffle(ah)

    if len(ah) >= ae:
        af = ah[:ae]
    else:
        af = ah.copy()
        al = [am for am in h if am not in af and am.lower() != ad.lower()]
        random.shuffle(al)
        if al:
            af.extend(al[:ae - len(af)])

    return af[:ae]

def an(ao, ap, aq):
    ar = ac(ao, 4)
    as_all = [ao] + ar

    correct_index = random.randint(0, 4)
    if as_all[0] == ao:
        as_all[0], as_all[correct_index] = as_all[correct_index], as_all[0]
    else:
        for i, val in enumerate(as_all):
            if val == ao:
                as_all[i], as_all[correct_index] = as_all[correct_index], as_all[i]
                break

    if ap not in g:
        g[ap] = {}
    g[ap][aq] = {
        'at': ao,
        'au': as_all
    }
    av = []
    for aw in range(len(as_all)):
        av.append([InlineKeyboardButton(
            f"Вариант {aw+1}",
            callback_data=f"ans_{ap}_{aq}_{aw}"
        )])
    ax = "\n".join([f"{ay+1}. {az}" for ay, az in enumerate(as_all)])
    return InlineKeyboardMarkup(av), as_all, ax

async def ba(bb: Update, bc: ContextTypes.DEFAULT_TYPE):
    bd = await bb.message.photo[-2].get_file() if len(bb.message.photo) > 1 else await bb.message.photo[-1].get_file()
    be = await bd.download_as_bytearray()
    bf = u(be)
    bg = bb.effective_user.id
    bh = await bb.message.reply_text("Обработка...")
    bi = bh.message_id
    bj, bk, bl = an(bf, bg, bi)
    bm = f"Выбери вариант:\n\n{bl}"
    await bh.edit_text(bm, reply_markup=bj)

async def bo(bp: Update, bq: ContextTypes.DEFAULT_TYPE):
    br = bp.callback_query
    await br.answer()
    bs = br.data.split('_')
    if len(bs) != 4:
        return
    bt = int(bs[1])
    bu = int(bs[2])
    bv = int(bs[3])
    if bt not in g or bu not in g[bt]:
        await br.edit_message_text("Сессия устарела.")
        return
    bw = g[bt][bu]
    bx = bw['at']
    by = bw['au']
    bz = by[bv]
    ca = (bz == bx)
    cb = []
    for cc, cd in enumerate(by):
        ce = "✓ " if cd == bx else ("✗ " if cc == bv and not ca else "  ")
        cb.append(f"{ce}{cc+1}. {cd}")
    cf = "\n".join(cb)
    if ca:
        cg = f"Верно.\n\n{cf}"
    else:
        cg = f"Неверно. Правильный вариант отмечен.\n\n{cf}"
    await br.edit_message_text(cg)
    del g[bt][bu]

ci = Application.builder().token(a).build()
ci.add_handler(CommandHandler("start", i))
ci.add_handler(CommandHandler("help", m))
ci.add_handler(CommandHandler("game", q))
ci.add_handler(MessageHandler(filters.PHOTO, ba))
ci.add_handler(CallbackQueryHandler(bo, pattern="^ans_"))
ci.run_polling()

Loading weights:   0%|          | 0/473 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie text_decoder.cls.predictions.bias to text_decoder.cls.predictions.decoder.bias, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-base
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identic

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


RuntimeError: Cannot close a running event loop